In [ ]:
import math
import random
import matplotlib.pyplot as plt

# Initialize parameters and jobs
mu = 1.0
job_cases = [
    {'coupling': 'complex', 'theta': -47, 'w1': 5.0, 'w2': 5.0, 'A12': 0.5, 'A21': 0.5},
    {'coupling': 'complex', 'theta': 98, 'w1': 5.0, 'w2': 5.0, 'A12': 0.5, 'A21': 0.5},
    {'coupling': 'power', 'theta': -47, 'w1': 5.0, 'w2': 15.0, 'A12': 0.5, 'A21': 0.5},
    {'coupling': 'power', 'theta': 98, 'w1': 5.0, 'w2': 15.0, 'A12': 0.5, 'A21': 0.5} ]

def simulate_osc(r1_0, r2_0, phi1_0, phi2_0, dt, num_steps, coupling, w1, w2, A12, A21, theta_rad):
    # Initialize arrays for storing results
    r1_values = [r1_0]
    r2_values = [r2_0]
    phi1_values = [phi1_0]
    phi2_values = [phi2_0]

    # Integration loop
    for i in range(num_steps - 1):
        r1 = r1_values[-1]
        r2 = r2_values[-1]
        phi1 = phi1_values[-1]
        phi2 = phi2_values[-1]

        # Initialize Derivatives based on coupling type
        if coupling == "complex":
            A = A12
            w = w1
            del_p1 = w + A * (r2 / r1) * math.sin(phi2 - phi1 + theta_rad)
            del_p2 = w + A * (r1 / r2) * math.sin(phi1 - phi2 - theta_rad)
            del_r1 = mu * (1 - r1**2) * r1 + A * r2 * math.cos(phi2 - phi1 + theta_rad)
            del_r2 = mu * (1 - r2**2) * r2 + A * r1 * math.cos(phi1 - phi2 - theta_rad)

        elif coupling == "power":
            del_p1 = w1 + A12 * (r2**(w1 / w2) / r1) * math.sin(w1 * ((phi2 / w2) - (phi1 / w1) + theta_rad / (w1 * w2)))
            del_p2 = w2 + A21 * (r1**(w2 / w1) / r2) * math.sin(w2 * ((phi1 / w1) - (phi2 / w2) + theta_rad / (w1 * w2)))
            del_r1 = (mu - r1**2) * r1 + A12 * (r2**(w1 / w2)) * math.cos(w1 * ((phi2 / w2) - (phi1 / w1) + theta_rad / (w1 * w2)))
            del_r2 = (mu - r2**2) * r2 + A12 * (r1**(w2 / w1)) * math.cos(w1 * ((phi1 / w1) - (phi2 / w2) + theta_rad / (w1 * w2)))

        else:
            raise ValueError("Invalid coupling type. Must be 'complex' or 'power'.")

        # Update values using Euler method
        r1_values.append(r1 + del_r1 * dt)
        r2_values.append(r2 + del_r2 * dt)
        phi1_values.append(phi1 + del_p1 * dt)
        phi2_values.append(phi2 + del_p2 * dt)

    return r1_values, r2_values, phi1_values, phi2_values

def plot_results(total_time, dt, job_cases):
    # Initialize random seeds and parameters
    random.seed(1000)
    r1_0 = random.random()
    r2_0 = random.random()
    p1_0 = random.uniform(-math.pi, math.pi)
    p2_0 = random.uniform(-math.pi, math.pi)

    num_steps = int(total_time / dt) + 1
    time_array = [i * dt for i in range(num_steps)]

    for case in job_cases:
        theta_rad = math.radians(case['theta'])
        r1, r2, p1, p2 = simulate_osc(
            r1_0, r2_0, p1_0, p2_0, dt, num_steps,
            case['coupling'], case['w1'], case['w2'],
            case['A12'], case['A21'], theta_rad
        )

        # Calculate phase difference based on coupling type
        if case['coupling'] == 'power':
            delta_p = [(p1_val / case['w1']) - (p2_val / case['w2']) + (theta_rad / (case['w1'] * case['w2'])) for p1_val, p2_val in zip(p1, p2)]
            phase_diff_label = r'$\frac{\phi_2}{\omega_2} - \frac{\phi_1}{\omega_1} + \frac{\theta}{\omega_1 \cdot \omega_2}$ (degrees)'
        else:
            delta_p = [(p1_val - p2_val - theta_rad) for p1_val, p2_val in zip(p1, p2)]
            phase_diff_label = r'$\phi_1 - \phi_2 - \theta$ (degrees)'

        # Normalize delta_p to be within [-π, π]
        delta_p = [math.degrees((dp + math.pi) % (2 * math.pi) - math.pi) for dp in delta_p]

        # Calculate radius projections for plotting
        r1_cosp1 = [r * math.cos(p) for r, p in zip(r1, p1)]
        r2_cosp2 = [r * math.cos(p) for r, p in zip(r2, p2)]

        # Plot phase difference
        plt.figure(figsize=(10, 5))
        plt.title(f"Phase Difference for {case['coupling'].capitalize()} Coupling, θ = {case['theta']}°")
        plt.plot(time_array, delta_p, label='Phase Difference')
        plt.xlabel("Time (seconds)")
        plt.ylabel(phase_diff_label)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"phase_diff {case['coupling']} - {case['theta']}", dpi=900)
        plt.show()
        
        # Plot radius projections
        plt.figure(figsize=(10, 5))
        plt.title(f"Radius Projection for {case['coupling'].capitalize()} Coupling, θ = {case['theta']}°")
        plt.plot(time_array, r1_cosp1, label='Oscillator 1 Projection')
        plt.plot(time_array, r2_cosp2, label='Oscillator 2 Projection')
        plt.xlabel("Time (seconds)")
        plt.ylabel(r'$R \cos(\text{phase})$')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"rad_proj {case['coupling']} - {case['theta']}", dpi=900)
        plt.show()


# Run the simulations and plot the results
plot_results(total_time=10, dt=0.0001, job_cases=job_cases)


In [ ]:
import math
import matplotlib.pyplot as plt

# Define parameters for the simulation
mu = 1.0
w1, w2 = 5.0, 15.0
A12, A21 = 0.5, 0.5
theta_complex = 47  # Degree of phase difference for complex coupling
theta_power = 0  # Degree of phase difference for power coupling

# Generate 20 initial phase points between -4 and 10 radians
initial_phases = [-4 + i * (14 / 19) for i in range(20)]

# Create job cases for complex and power coupling with different initial phases
complex_job_cases = [{'coupling': 'complex', 'theta': theta_complex, 'w1': w1, 'w2': w1, 'A12': A12, 'A21': A21, 
                      'initial_phase1': -4, 'initial_phase2': phase + -4} for phase in initial_phases]
power_job_cases = [{'coupling': 'power', 'theta': theta_power, 'w1': w1, 'w2': w2, 'A12': A12, 'A21': A21, 
                    'initial_phase1': -4 *w1, 'initial_phase2': (phase + -4)*w2} for phase in initial_phases]

def plot_convergence(total_time, dt, job_cases, coupling_type):
    """
    Plots convergence for a series of initial conditions for a specified coupling type.
    """
    num_steps = int(total_time / dt) + 1
    time_array = [i * dt for i in range(num_steps)]
    
    plt.figure(figsize=(12, 6))
    for case in job_cases:
        theta_rad = math.radians(case['theta'])
        r1_0, r2_0 = 1.0, 1.0  # Start with fixed amplitudes
        p1_0, p2_0 = case['initial_phase1'], case['initial_phase2']
        
        # Run the simulation
        r1_values, r2_values, p1_values, p2_values = simulate_osc(
            r1_0, r2_0, p1_0, p2_0, dt, num_steps,
            case['coupling'], case['w1'], case['w2'],
            case['A12'], case['A21'], theta_rad
        )
        
        # Calculate the phase difference or normalized phase difference
        if case['coupling'] == 'power':
            delta_p = [(p1 / case['w1']) - (p2 / case['w2']) + (theta_rad / (case['w1'] * case['w2'])) for p1, p2 in zip(p1_values, p2_values)]
            phase_label = r'$\frac{\phi_2}{\omega_2} - \frac{\phi_1}{\omega_1} + \frac{\theta}{\omega_1 \cdot \omega_2}$'
        else:
            delta_p = [(p1 - p2 - theta_rad) for p1, p2 in zip(p1_values, p2_values)]
            phase_label = r'$\phi_1 - \phi_2 - \theta$'
        
        # Normalize delta_p to be within [-3π, 3π]
        delta_p_radians = [(dp + 3 * math.pi) % (6 * math.pi) - 3 * math.pi for dp in delta_p]
        
        # Plot convergence of phase difference over time for each initial condition
        plt.plot(time_array, delta_p_radians)
    
    plt.xlabel("Time (seconds)")
    plt.ylabel(f"Phase Difference {phase_label} (radians)")
    plt.title(f"Convergence of Phase Difference for {coupling_type.capitalize()} Coupling")
    plt.grid(True)

    # Add horizontal lines at multiples of π
    for i in range(-3, 4):
        plt.axhline(y=i * math.pi, color='gray', linestyle='--', linewidth=0.5)
        plt.text(time_array[-1] * 1.01, i * math.pi, f"{i}π", va='center', ha='left', color='gray')

    plt.tight_layout()
    plt.savefig(f"rand_start - {case['coupling']}")
    plt.show()

# Run and plot convergence for complex coupling
plot_convergence(total_time=10, dt=0.0001, job_cases=complex_job_cases, coupling_type="complex")

# Run and plot convergence for power coupling
plot_convergence(total_time=10, dt=0.0001, job_cases=power_job_cases, coupling_type="power")
